In [1]:
import xclim as xc
import xarray as xr
import _pickle as cpickle
import numpy as np
import re

In [2]:
import sys
sys.path.append('../scripts/')
sys.path.append("/home/abhi/Documents/mygit/postBC_diagnostic/src/scripts")
from recipes import *

In [3]:
def get_seasgrpr(ds):
    seasgrpr = xr.DataArray(pd.Series(ds['time.month'].values)
                            .apply(getseas), dims={'time':ds['time']},
                            name='seas')
    
    return(seasgrpr)

In [4]:
seasIndex = pd.Index(['Annual', 'DJF', 'MAM', 'JJAS', 'ON'], name='seas')

### Load the dataset

In [6]:
with open('../../pickles/Amravati/Amravati_ALL21_rcp45_ds.pkl', 'rb') as f:
    ds = cpickle.load(f)

In [7]:
with open('../../pickles/Amravati/Amravati_ALL21_historical_ds.pkl', 'rb') as f:
    ds_base = cpickle.load(f)

### Extract out the precipitation dataarray and multiply with 86400

In [9]:
pr = ds.sel(variable='pr')['value']*86400
pr.attrs['units'] = 'mm/day'

In [28]:
pr_base = ds_base.sel(variable='pr')['value']*86400
pr_base.attrs['units'] = 'mm/day'

In [7]:
pr

<xarray.DataArray 'value' (model: 21, time: 34310)>
array([[0.      , 0.      , 0.      , ..., 0.      , 0.      , 0.      ],
       [0.      , 0.      , 0.      , ..., 0.      , 0.      , 0.      ],
       [0.      , 0.      , 0.      , ..., 0.      , 0.      , 0.      ],
       ...,
       [0.      , 0.      , 0.      , ..., 0.      , 0.      , 0.      ],
       [0.      , 0.      , 0.      , ..., 0.      , 0.      , 0.      ],
       [0.      , 0.      , 0.      , ..., 0.      , 0.      , 0.010201]])
Coordinates:
  * time      (time) datetime64[ns] 2006-01-01T12:00:00 ... 2099-12-31T12:00:00
    variable  <U2 'pr'
  * model     (model) object 'ACCESS1-0' 'BCC-CSM1-1' ... 'NorESM1-M'
Attributes:
    units:    mm/day

### Indices calculation

#### intensity_above_thresh

In [8]:
res = xc.indices.daily_pr_intensity(pr, thresh='1 mm/day', freq='MS')

In [60]:
def yearseasmean(ds):
    res = xr.concat([yearmean(ds)] + [yearmean(selseas(ds, seas)) for seas in seasIndex[1:]],
                   dim=seasIndex)
    
    return(res)

In [61]:
def intensity_above_thresh(ds, thresh='1 mm/day', freq='MS'):
    res = yearseasmean(xc.indices.daily_pr_intensity(pr, thresh='1 mm/day', freq='MS'))
    return(res)
    

In [62]:
intensity_above_thresh(pr)

<xarray.DataArray 'value' (seas: 5, model: 21, year: 94)>
array([[[13.511349, 10.553436, ...,  6.013226, 10.304479],
        [ 7.913057,  8.035091, ...,  6.855527,  6.578495],
        ...,
        [20.619035, 11.544139, ..., 19.389004, 24.850097],
        [ 7.539294,  6.892458, ..., 13.730077, 13.337861]],

       [[      nan,  2.1247  , ...,       nan,       nan],
        [11.651332,  1.234272, ...,  1.671188,  5.55951 ],
        ...,
        [25.492444,  3.511683, ...,       nan,  2.0777  ],
        [ 1.921563,       nan, ...,       nan, 16.974394]],

       ...,

       [[ 6.851316, 12.465498, ...,  5.933042, 13.951252],
        [10.287418,  9.743216, ...,  9.483839,  8.00358 ],
        ...,
        [21.429355, 12.84127 , ..., 10.33046 , 23.923862],
        [10.208393,  9.98719 , ..., 12.224537, 12.407865]],

       [[ 8.136365, 19.030672, ..., 10.840801,       nan],
        [ 4.426489,  8.019253, ...,  1.526617,  4.74731 ],
        ...,
        [ 3.230799,  5.845129, ...,  9.362282

#### days_above_thresh

In [12]:
res = xc.indices.wetdays(pr, thresh='1 mm/day', freq='MS')

In [13]:
def yearsum(xr_ds):
    return xr_ds.groupby('time.year').sum('time')

In [14]:
def yearseassum(ds):
    res = xr.concat([yearsum(ds)] + [yearsum(selseas(ds, seas)) for seas in seasIndex[1:]],
                   dim=seasIndex)
    
    return(res)

In [63]:
def days_above_thresh(ds, thresh='1 mm/day', freq='MS'):
    res = yearseassum(xc.indices.wetdays(pr, thresh='1 mm/day', freq='MS'))
    return(res)

In [64]:
days_above_thresh(pr)

<xarray.DataArray 'value' (seas: 5, model: 21, year: 94)>
array([[[57, 37, ..., 56, 76],
        [89, 79, ..., 86, 99],
        ...,
        [94, 38, ..., 27, 54],
        [70, 83, ..., 65, 65]],

       [[ 0,  2, ...,  0,  0],
        [ 1,  1, ...,  1,  4],
        ...,
        [ 2,  1, ...,  0,  2],
        [ 1,  0, ...,  0,  1]],

       ...,

       [[53, 26, ..., 51, 70],
        [82, 71, ..., 76, 86],
        ...,
        [89, 34, ..., 23, 50],
        [58, 78, ..., 56, 59]],

       [[ 1,  7, ...,  1,  0],
        [ 2,  7, ...,  9,  9],
        ...,
        [ 2,  1, ...,  3,  1],
        [ 9,  3, ...,  9,  3]]])
Coordinates:
    variable  <U2 'pr'
  * model     (model) object 'ACCESS1-0' 'BCC-CSM1-1' ... 'NorESM1-M'
  * year      (year) int64 2006 2007 2008 2009 2010 ... 2095 2096 2097 2098 2099
  * seas      (seas) object 'Annual' 'DJF' 'MAM' 'JJAS' 'ON'

#### intensity_above_pctl and days_above_pctl

Ways of calculating pctl:
    1) Whole time series
    2) Month wise
    3) Season wise
    4) Day of year wise

In [16]:
def groupby(ds, method='whole'):
    if method == 'whole':
        ds = ds
    elif method == 'month':
        ds = ds.groupby('time.month')
    elif method == 'doy':
        ds = ds.groupby('time.dayofyear')
    elif method == 'seas':
        ds = groupbyseas(ds)
        
    return(ds)

In [17]:
def calc_pctl(ds, q, method='whole'):
    ds = groupby(ds, method=method)
    res = ds.reduce(np.percentile, q=q, dim='time')
        
    return(res)

In [18]:
def intensity_and_days_above_pctl(ds_fut, ds_base, pctl):
    qseas = calc_pctl(ds_base, pctl, method='seas')
    qwhole = calc_pctl(ds_base, pctl, method='whole')
    
    q = xr.concat([qwhole] + [qseas.sel(seas=seas).drop('seas') for seas in seasIndex[1:]],
          dim=seasIndex)
    
    results = {'intensity': [],
               'days': []}
    
    for seas in seasIndex:
        anom  = selseas(ds_fut, seas).groupby('model') - q.sel(seas=seas)
        anom.attrs['units'] = 'mm/day'
        
        results['days'].append(xc.indices.wetdays(anom, '0 mm/day', freq='YS'))
        results['intensity'].append(xc.indices.daily_pr_intensity(anom, '0 mm/day', freq='YS'))
        
    
    for k, dsets in results.items():
        results[k] = xr.concat(dsets, dim=seasIndex)
    
    
    return(results)
    

In [55]:
res = intensity_and_days_above_pctl(pr, selyear(pr_base, range(1976, 2006)), 90)['intensity'] -\
intensity_and_days_above_pctl(selyear(pr_base, range(1976, 2006)), selyear(pr_base, range(1976, 2006)), 90)['intensity'].mean(dim='time')

In [56]:
selyear(res, range(2021, 2051)).mean(dim=['model', 'time'])


<xarray.DataArray 'value' (seas: 5)>
array([ 1.362235, -0.002647,  0.014557,  1.731596,  0.794534])
Coordinates:
    variable  <U2 'pr'
  * seas      (seas) object 'Annual' 'DJF' 'MAM' 'JJAS' 'ON'

In [114]:
(res>0).sum(dim='time')

<xarray.DataArray 'value' (model: 21)>
array([342, 342, 342, 342, 342, 342, 330, 342, 342, 342, 342, 342, 342, 342,
       342, 342, 311, 342, 342, 289, 342])
Coordinates:
    variable  <U2 'pr'
    seas      <U2 'ON'
  * model     (model) object 'ACCESS1-0' 'BCC-CSM1-1' ... 'NorESM1-M'

####  max_n_day_pr_amt 

In [118]:
def max_n_day_pr_amt(ds, ndays, **kwargs):
    results = {}
    
    for seas in seasIndex:
        results[seas] = xc.indices.max_n_day_precipitation_amount(selseas(ds, seas),
                                                                          window=ndays, **kwargs)
        
    res = xr.concat(results.values(), dim=seasIndex)
    
    return(res)

In [123]:
max_n_day_pr_amt(pr, 5).sel(seas='Annual')

<xarray.DataArray (model: 21, time: 56)>
array([[109.916621, 171.354839, 218.681859, ..., 152.910217, 148.812673,
        241.25679 ],
       [138.997967, 197.483399,  87.328119, ..., 230.881098, 102.760961,
        141.381561],
       [ 75.485457,  94.347773, 128.351308, ..., 186.93602 ,  78.072789,
         85.822667],
       ...,
       [153.227272, 137.366605, 174.670017, ..., 149.235789, 103.98921 ,
        272.738347],
       [189.57892 ,  77.89276 , 137.961795, ..., 560.145212,  77.979259,
        199.927501],
       [184.136193, 233.912748, 223.385225, ...,  71.106089, 133.451544,
         95.20555 ]])
Coordinates:
  * time      (time) datetime64[ns] 1950-01-01 1951-01-01 ... 2005-01-01
    variable  <U2 'pr'
  * model     (model) object 'ACCESS1-0' 'BCC-CSM1-1' ... 'NorESM1-M'
    seas      <U6 'Annual'
Attributes:
    units:    mm

#### max_cdd and max_cwd

In [126]:
def max_consec_days(ds, metric, thresh='1 mm/day', **kwargs):
    
    if metric == 'cdd':
        func = xc.indices.maximum_consecutive_dry_days
    elif metric == 'cwd':
        func = xc.indices.maximum_consecutive_wet_days
        
    
    results = {}
    
    for seas in seasIndex:
        results[seas] = func(selseas(ds, seas), thresh=thresh, **kwargs)
        
    res = xr.concat(results.values(), dim=seasIndex)
    
    return(res)

In [128]:
max_consec_days(pr, 'cwd')

<xarray.DataArray 'value' (seas: 5, model: 21, time: 56)>
array([[[24, 28, ..., 19, 37],
        [39, 17, ..., 19, 14],
        ...,
        [17, 12, ..., 10, 19],
        [23, 15, ..., 22, 20]],

       [[ 0,  1, ...,  1,  1],
        [ 0,  0, ...,  1,  1],
        ...,
        [ 1,  1, ...,  0,  1],
        [ 0,  2, ...,  2,  2]],

       ...,

       [[24, 28, ..., 19, 37],
        [39, 17, ..., 19, 14],
        ...,
        [17, 12, ..., 10, 19],
        [23, 15, ..., 22, 20]],

       [[ 1,  3, ...,  1,  4],
        [ 3,  3, ...,  3,  2],
        ...,
        [ 1,  0, ...,  1,  1],
        [ 2,  0, ...,  4,  3]]])
Coordinates:
  * time      (time) datetime64[ns] 1950-01-01 1951-01-01 ... 2005-01-01
    variable  <U2 'pr'
  * model     (model) object 'ACCESS1-0' 'BCC-CSM1-1' ... 'NorESM1-M'
  * seas      (seas) object 'Annual' 'DJF' 'MAM' 'JJAS' 'ON'
Attributes:
    units:    days

#### cdd and cwd

In [194]:
def get_thresh_val_from_str(thresh):
    p = re.compile(r'([\-]?[0-9]+[\.[0-9]?[0-9]?]?).')
    thresh_val = float(p.search(thresh).groups()[0])
    
    return(thresh_val)

In [184]:
def num_consec_days(ds, metric, thresh='1 mm/day', ndays=5):
    thresh_val = get_thresh_val_from_str(thresh)
    
    if metric == 'cdd':
        cond = ds < thresh_val
    elif metric == 'cwd':
        cond = ds > thresh_val
        
    results = {}
    
    for seas in seasIndex:
        results[seas] = (selseas(cond, seas)
                         .groupby('time.year')
                         .apply(xc.run_length.windowed_run_count, window=ndays)
                        )
        
    
    res = xr.concat(results.values(), dim=seasIndex)
    
    return(res)

    

In [207]:
num_consec_days(pr, 'cwd', thresh='1 mm/day', ndays=5)

<xarray.DataArray 'value' (seas: 5, model: 21, year: 56)>
array([[[46, 87, ..., 40, 76],
        [95, 65, ..., 61, 50],
        ...,
        [47, 20, ..., 27, 40],
        [60, 22, ..., 43, 65]],

       [[ 0,  0, ...,  0,  0],
        [ 0,  0, ...,  0,  0],
        ...,
        [ 0,  0, ...,  0,  0],
        [ 0,  0, ...,  0,  0]],

       ...,

       [[46, 86, ..., 40, 76],
        [95, 65, ..., 61, 50],
        ...,
        [47, 20, ..., 27, 40],
        [60, 22, ..., 43, 65]],

       [[ 0,  0, ...,  0,  0],
        [ 0,  0, ...,  0,  0],
        ...,
        [ 0,  0, ...,  0,  0],
        [ 0,  0, ...,  0,  0]]])
Coordinates:
    variable  <U2 'pr'
  * model     (model) object 'ACCESS1-0' 'BCC-CSM1-1' ... 'NorESM1-M'
  * year      (year) int64 1950 1951 1952 1953 1954 ... 2001 2002 2003 2004 2005
  * seas      (seas) object 'Annual' 'DJF' 'MAM' 'JJAS' 'ON'